In [10]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import sys
import os
import shutil
from PIL import Image, ImageFont, ImageDraw
import cv2

# 添加路径，以能正常导入mbsh、trainer
sys.path.append(r'..\Lib\trainer')

from mbsh.core.yolo import YOLO
import numpy as np
import pandas as pd
import glob

2022-11-01 16:55:56,849 - DEBUG - pyplot.py - switch_backend - 280 - Loaded backend module://matplotlib_inline.backend_inline version unknown.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

### 公共函数

In [7]:
delta1 = 4       # 用于合并间断帧
delta2 = 2 * 18  # 用于筛选最终息肉
fps = 18

# 统计一个视频模型标记息肉开始结束帧
def stat_ct_file(file_path):
    ct, start, end = 0, 0, 0
    results = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            arr = line.strip().split(',')
            if arr[1] == '1':
                if start == 0:
                    start = int(arr[0][:-4])
            else:
                if start != 0:
                    end = int(arr[0][:-4])
                    if end - start >= delta1:
                        results.append([start, end])
                    start = 0
                    
        return results


def stat_ct(data):
    ct, start, end = 0, 0, 0
    results = []
    for arr in data:
        if arr[1] == 1:
            if start == 0:
                start = int(arr[0][:-4])
        else:
            if start != 0:
                end = int(arr[0][:-4])
                if end - start >= delta1:
                    results.append([start, end])
#                 results.append([start, end])
                start = 0
                    
    return results


# 合并一个视频模型标记息肉开始结束帧
def merge_ct(arr):
    if len(arr) == 0:
        return []
    
    results = [arr[0]]
    for idx in range(1, len(arr)):
        if arr[idx][0] - results[-1][1]  < delta1:
            results[-1][1] = arr[idx][1]
        else:
            results.append(arr[idx])
            
    return results

# 按照长度过滤息肉，>= 2S的才选择
def filt_polyp(arr):
    results = []
    for start, end in arr:
        if end - start >= delta1 - 1:
            results.append([start, end])
    
    return results
    

# 将目录的图片名称按照名称数字从小到大排序
def get_dir_sorted_imgs(dirs):
    names = []
    if isinstance(dirs, str):
        if os.path.exists(dirs):
            names = [int(filename[:-4]) for filename in os.listdir(dirs) if filename.endswith('.jpg')]
    elif isinstance(dirs, list):
        for dir in dirs:
            if os.path.exists(dir):
                names.extend([int(filename[:-4]) for filename in os.listdir(dir) if filename.endswith('.jpg')])
    names = sorted(names)
    img_names = ['%s.jpg' % name for name in names]
    return img_names


### 息肉检测模型

In [4]:
model_path = r'D:\projects\Vision\polyp\Models\trained_weights_final.h5'
anchors_path = r'D:\projects\Vision\polyp\Models\yolo_anchors_0311.txt'
classes_path = r'D:\projects\Vision\polyp\Models\polyp_classes.txt'
iou = 0.1

model_image_size = (352, 352)
gpu_num = 1 

In [5]:
# 模型检测标记息肉
def detect_polyp_by_model(input_path, target_path, score):
    """
    input_path：输入视频图片目录
    target_path：输出标记图片目录
    score：模型参数
    """
    if not os.path.exists(target_path):
        os.mkdir(target_path)

    # 初始化模型
    yolo = YOLO(model_path=model_path, anchors_path=anchors_path, classes_path=classes_path,
               score=score, iou=iou, model_image_size=model_image_size, gpu_num=gpu_num)

    # 模型预测
    results = []
    img_names = get_dir_sorted_imgs(input_path)
    for filename in img_names:
        img_path = os.path.join(input_path, filename)
        pred_results, image, c = yolo.predict_file(img_path, target_path, draw_rect=True, cut=False, 
                                                         one_box=True, timing=False, expand=1)
        if len(pred_results) == 1:
            #results.append('%s,%s,%s,%s,%s,%s,%s\n' % (filename, 1, pred_results[0][4], pred_results[0][0], pred_results[0][1],pred_results[0][2], pred_results[0][3]))
            results.append((filename, 1))
        else:
            results.append((filename, 0))

    return results

# 息肉统计结果写入文件
def write_stat_result(output_file, video_name, score, poly_results, video_len, fps):
    # 息肉个数
    r = stat_ct(poly_results)
    r = merge_ct(r)
#     r = filt_polyp(r)
    poly_num = len(r)
    
    with open(output_file, 'a+') as f:
    # 写息肉出现时间点
        for rr in r:
            f.write('%.3f	%d	%d\n' % (rr[0] / fps, rr[0], rr[1]))
    
    # 写息肉个数和视频长度比
        f.write('%s	%.3f	%.3f\n' % (video_name, score, poly_num/video_len))
        
    return poly_num/video_len
        
# 根据视频图片数量统计视频长度
def get_video_len(video_imgs_dir, fps):
    imgs = list(os.listdir(video_imgs_dir))
    return len(imgs) / (fps * 60)
    

### 挑选真息肉

In [7]:
import shutil
base_dir = r'D:\项目资料\息肉假阳性\20221007'
from_path = os.path.join(base_dir, 'images_crop')
to_path = os.path.join(base_dir, 'images_truepoly_crop')

folder_imgs = [(1006, [(106, 474), (3211, 3294)]), (1014, [(5115, 5160), (5591, 5681)]), (1037, [(1441, 1525)]), (1067, [(5122, 5247)]), 
                (1087, [(4672, 5048)]), (1112, [(5806, 5880)]), (1156, [(5825, 5909)]), (1255, [(4653, 4688), (6179, 6216)]), 
                (1264, [(3157, 3220)]), (1293, [(1303, 1470)]), (1307, [(1131, 1183)]), (1435, [(5616, 5707)]), (1447, [(4743, 4879)])]
for folder, start_ends in folder_imgs:
    from_folder = os.path.join(from_path, str(folder))
    to_folder = os.path.join(to_path, str(folder))
    print('processing: %s' % folder)
    if not os.path.exists(to_folder):
        os.mkdir(to_folder)
        
    for start, end in start_ends:
        for i in range(start, end+1):
            img_name = '%s.jpg' % i
            from_img = os.path.join(from_folder, img_name)
            shutil.copy(from_img, os.path.join(to_folder, img_name))
            if os.path.exists(from_img):
                os.remove(from_img)
              


processing: 1006
processing: 1014
processing: 1037
processing: 1067
processing: 1087
processing: 1112
processing: 1156
processing: 1255
processing: 1264
processing: 1293
processing: 1307
processing: 1435
processing: 1447


In [22]:
a = [(106, 474), (3211, 3294)]
a[0][1] - a[0][0] + a[1][1] - a[1][0]

451

### 运行息肉检测部分

In [7]:
fps = 18
# score = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35]
base_dir = r'D:\项目资料\息肉假阳性\20221007'

## 假阳性检测
video_names_score=[
#                     ('3398', 0.015), # 12.63
#                     ('3398', 0.05), # 9.02
#                     ('3399', 0.01), # 11.2
#                     ('3399', 0.015), # 8.826
#                     ('3402', 0.015), # 10.42
#                     ('3402', 0.03), # 7.42
#                     ('3414', 0.01), # 10.54
#                     ('3414', 0.015), # 9.17
#                     ('3415', 0.01), # 11.44
#                     ('3415', 0.03), # 6.45
#                     ('3416', 0.005), # 12.99
#                     ('3416', 0.015), # 5.35
#                     ('3419', 0.005), # 13.44
#                     ('3419', 0.015), # 6.84
#                     ('3429', 0.015), # 13.8
#                     ('3429', 0.03), # 9.14
#                     ('3434', 0.015),  # 10.11
#                     ('3434', 0.03), # 8.052
#                     ('3435', 0.015), # 7.46
#                     ('3435', 0.005) # 13.44
]

# video_names_score=[('1006', 0.015)] # 0.03：8.84, 0.015: 13.95

input_base_dir = os.path.join(base_dir, 'images_crop_wxr')
output_base_dir = os.path.join(base_dir, 'images_detect_wxr')
for video_name, score in video_names_score:
    print(video_name, score)
    poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
                                         os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)
    val = write_stat_result(os.path.join(base_dir, 'stat.txt'), video_name, score, poly_results,
                      get_video_len(os.path.join(input_base_dir, video_name), fps), fps)
    print(val)
                  

2022-10-29 23:04:01,437 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


3398 0.05
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-29 23:25:01,929 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


9.02339398440401
3399 0.01
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-30 00:13:48,674 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


...

11.44729234920023
3416 0.005
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-30 03:07:41,746 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


12.991071428571429
3419 0.005
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-30 04:04:05,166 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


13.44708323726078
3435 0.005
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.
13.44212418752593


In [10]:
## 真阳性检测
true_poly_score = 0.001
input_base_dir = os.path.join(base_dir, 'images_truepoly_crop')
output_base_dir = os.path.join(base_dir, 'images_truepoly_detect')
for video_name in os.listdir(input_base_dir):
    score = true_poly_score
    print(video_name, score)
    poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
                                         os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)

2022-10-13 14:05:29,636 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1006 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-13 14:06:35,559 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1014 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-13 14:06:56,716 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


...

1307 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-13 14:09:52,892 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1435 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


2022-10-13 14:10:07,623 - INFO - yolo.py - generate - 132 - load yolo model D:\projects\Vision\polyp\Models\trained_weights_final.h5


1447 0.001
<class 'keras.engine.functional.Functional'> 111111111
D:\projects\Vision\polyp\Models\trained_weights_final.h5 model, anchors, and classes loaded.


### 运行真息肉标记
暂时不用

In [17]:
input_base_dir = os.path.join(base_dir, 'images_crop')
output_base_dir = os.path.join(base_dir, 'images_detect')

# 找到文件夹对应的score后缀
folder_score = {}
for folder in os.listdir(output_base_dir):
    arr = folder.split('_')
    folder_score[arr[0]] = arr[1]
    
for folder, score in folder_score.items():
    if folder != '3531':
        continue
    print(folder, score)
    results = []
    img_names = get_dir_sorted_imgs(os.path.join(input_base_dir, folder))
    det_img_names = set(get_dir_sorted_imgs(os.path.join(output_base_dir, '%s_%s' % (folder, score))))
    for img_name in img_names:
        if img_name not in det_img_names:
            results.append((img_name, 0))
        else:
            results.append((img_name, 1))
            
    val = write_stat_result(os.path.join(base_dir, 'stat.txt'), folder, float(score), results,
                      get_video_len(os.path.join(input_base_dir, folder), fps), fps)
    print(val)
    

## 假阳性设置score
# for video_name, score in [
#     ('3429', 0.008)
# ]:
## 真阳性检测
# for video_name in os.listdir(input_base_dir):
#     score = true_poly_score
    
#     print(video_name, score)
#     poly_results = detect_polyp_by_model(os.path.join(input_base_dir, video_name), 
#                                          os.path.join(output_base_dir, '%s_%s' % (video_name, score)), score)
#     val = write_stat_result(os.path.join(base_dir, 'stat.txt'), video_name, score, poly_results,
#                       get_video_len(os.path.join(input_base_dir, video_name), fps), fps)
#     print(val)
                  

In [ ]:
### 运行记录
3400 0.01 12.839
3400 0.35 3.959

### 合成视频

In [8]:
import os
import numpy as np
import cv2
fps = 18

# 将生成的息肉图片和原始图片一起合成息肉标记视频
def create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path):
    
    image = cv2.imdecode(np.fromfile(os.path.join(raw_img_paths[0], os.listdir(raw_img_paths[0])[0]), dtype=np.uint8), -1)
    img_h, img_w, _ = image.shape

    video_path = output_video_path
    video = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*"XVID"), fps, (img_w, img_h))
    
    img_names = get_dir_sorted_imgs(raw_img_paths)
    for f in img_names:
        filepath11 = os.path.join(raw_img_paths[0], f)
        filepath12 = os.path.join(raw_img_paths[1], f)
        filepath2 = os.path.join(fake_polyp_path, f)
        filepath3 = os.path.join(true_polyp_path, f)
        if os.path.exists(filepath3):
            filepath = filepath3
        elif os.path.exists(filepath2):
            filepath = filepath2
        elif os.path.exists(filepath11):
            filepath = filepath11
        elif os.path.exists(filepath12):
            filepath = filepath12
        image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8),-1)
        video.write(image)
    video.release()

In [9]:
## 无辅助视频合成
for base_dir in [r'D:\项目资料\息肉假阳性\20221007']:
    raw_img_folder = os.path.join(base_dir, 'images_crop_wfz')
    fake_polyp_folder = os.path.join(base_dir, 'images_detect_wfz')
    true_polyp_folder = os.path.join(base_dir, 'images_truepoly_detect_wfz')
    output_video_folder = os.path.join(base_dir, 'videos_gen_wfz')
    
    # 循环处理
    for video_name in ['3429', '3434', '3435', '3438', '3442', '3479', '3497', '3513', '3538']:
        print('processing: %s' % (video_name))
        raw_img_paths = [os.path.join(raw_img_folder, video_name), os.path.join(raw_img_folder, video_name)]
        fake_polyp_path = os.path.join(fake_polyp_folder, '%s' % (video_name))
        true_polyp_path = os.path.join(true_polyp_folder, '%s' % (video_name))
        output_video_path = os.path.join(output_video_folder, r'%s.avi' % (video_name))

        create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path)

processing: 3429
processing: 3434
processing: 3435
processing: 3438
processing: 3442
processing: 3479
processing: 3497
processing: 3513
processing: 3538


In [9]:
for base_dir in [r'D:\项目资料\息肉假阳性\20221007']:
    raw_img_folders = [os.path.join(base_dir, 'images_crop_wfz')]
    fake_polyp_folder = os.path.join(base_dir, 'images_detect_wfz')
    true_polyp_folder = os.path.join(base_dir, 'images_truepoly_detect_wfz')
    output_video_folder = os.path.join(base_dir, 'videos_gen_wfz')
    
    # 文件夹名和分数
#     video_names = os.listdir(raw_img_folder)
#     folder_score = {video_name: '' for video_name in video_names}
    folder_score_arr = []
    for folder in os.listdir(fake_polyp_folder):
        arr = folder.split('_')
        folder_score_arr.append((arr[0], arr[1]))
    
    # 循环处理
    for video_name, score in folder_score_arr:
        print('processing: %s, %s' % (video_name, score))
        raw_img_paths = [os.path.join(rf, video_name) for rf in raw_img_folders]
        fake_polyp_path = os.path.join(fake_polyp_folder, '%s_%s' % (video_name, score))
        true_polyp_path = os.path.join(true_polyp_folder, '%s_%s' % (video_name, 0.001))
        output_video_path = os.path.join(output_video_folder, r'%s_%s.avi' % (video_name, score))

        create_mp4(raw_img_paths, fake_polyp_path, true_polyp_path, output_video_path)
        

processing: 3398, 0.015
processing: 3398, 0.03
processing: 3398, 0.05
processing: 3399, 0.01
processing: 3399, 0.015
processing: 3399, 0.03
processing: 3402, 0.015
processing: 3402, 0.03
processing: 3414, 0.01
processing: 3414, 0.015
processing: 3414, 0.03
processing: 3415, 0.01
processing: 3415, 0.015
processing: 3415, 0.03
processing: 3416, 0.005
processing: 3416, 0.015
processing: 3416, 0.03
processing: 3419, 0.005
processing: 3419, 0.015
processing: 3419, 0.03
processing: 3429, 0.015
processing: 3429, 0.03
processing: 3434, 0.015
processing: 3434, 0.03
processing: 3435, 0.005
processing: 3435, 0.015
processing: 3435, 0.03
